In [1]:
import scanpy as sc
import numpy as np
import pandas as pd

In [2]:
import os
os.chdir('./../')
from cpa.helper import rank_genes_groups_by_cov

In [4]:
adatas = []
for i in range(5):
    adatas.append(sc.read(f'/data/share/cnic02/data/cpa_binaries/datasets/sciplex_raw_chunk_{i}.h5ad'))
adata = adatas[0].concatenate(adatas[1:])

In [5]:
sc.pp.subsample(adata, fraction=0.5)
sc.pp.normalize_per_cell(adata)

In [6]:
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=5000, subset=True)

In [8]:
adata.obs['dose_val'] = adata.obs.dose.astype(float) / np.max(adata.obs.dose.astype(float))
adata.obs['dose_val'][adata.obs['product_name'].str.contains('Vehicle')] = 1.0
adata.obs['product_name'] = [x.split(' ')[0] for x in adata.obs['product_name']]
adata.obs['product_name'][adata.obs['product_name'].str.contains('Vehicle')] = 'control'
adata.obs['drug_dose_name'] = adata.obs.product_name.astype(str) + '_' + adata.obs.dose_val.astype(str)
adata.obs['cov_drug_dose_name'] = adata.obs.cell_type.astype(str) + '_' + adata.obs.drug_dose_name.astype(str)
adata.obs['condition'] = adata.obs.product_name.copy()
adata.obs['control'] = [1 if x == 'Vehicle_1.0' else 0 for x in adata.obs.drug_dose_name.values]
adata.obs['cov_drug'] = adata.obs.cell_type.astype(str) + '_' + adata.obs.condition.astype(str)

In [9]:
from cpa.helper import rank_genes_groups_by_cov
rank_genes_groups_by_cov(adata, groupby='cov_drug', covariate='cell_type', control_group='control')

A549
MCF7
K562


In [10]:
new_genes_dict = {}
for cat in adata.obs.cov_drug_dose_name.unique():
    if 'control' not in cat:
        rank_keys = np.array(list(adata.uns['rank_genes_groups_cov'].keys()))
        bool_idx = [x in cat for x in rank_keys]
        genes = adata.uns['rank_genes_groups_cov'][rank_keys[bool_idx][0]]
        new_genes_dict[cat] = genes

In [11]:
adata.uns['rank_genes_groups_cov'] = new_genes_dict

# Split

In [12]:
adata.obs['split'] = 'train'  # reset
ho_drugs = [
    # selection of drugs from various pathways
    "Azacitidine",
    "Carmofur",
    "Pracinostat",
    "Cediranib",
    "Luminespib",
    "Crizotinib",
    "SNS-314",
    "Obatoclax",
    "Momelotinib",
    "AG-14361",
    "Entacapone",
    "Fulvestrant",
    "Mesna",
    "Zileuton",
    "Enzastaurin",
    "IOX2",
    "Alvespimycin",
    "XAV-939",
    "Fasudil"
]
ood = adata.obs['condition'].isin(ho_drugs)
len(ho_drugs)

19

In [13]:
adata.obs['split'][ood & (adata.obs['dose_val'] == 1.0)] = 'ood'
test_idx = sc.pp.subsample(adata[adata.obs['split'] != 'ood'], .10, copy=True).obs.index
adata.obs['split'].loc[test_idx] = 'test'

In [14]:
pd.crosstab(adata.obs['split'], adata.obs['condition'])

condition,(+)-JQ1,2-Methoxyestradiol,A-366,ABT-737,AC480,AG-14361,AG-490,AICAR,AMG-900,AR-42,...,Valproic,Vandetanib,Veliparib,WHI-P154,WP1066,XAV-939,YM155,ZM,Zileuton,control
split,,,,,,,,,,,,,,,,,,,,,
ood,0,0,0,0,0,355,0,0,0,0,...,0,0,0,0,0,249,0,0,403,0
test,168,141,170,133,165,137,189,173,149,156,...,192,142,152,180,190,120,40,134,127,664
train,1338,1346,1511,1261,1490,1162,1559,1650,1187,1250,...,1612,1225,1445,1584,1661,1057,354,1260,1198,5800


In [15]:
adata.obs['split'].value_counts()

split
train    256214
test      28468
ood        6206
Name: count, dtype: int64

In [16]:
adata[adata.obs.split == 'ood'].obs.condition.value_counts()

condition
Fasudil         474
Mesna           464
IOX2            444
Entacapone      433
Fulvestrant     417
Zileuton        403
Azacitidine     385
Carmofur        379
Enzastaurin     366
AG-14361        355
Pracinostat     318
SNS-314         280
Crizotinib      256
XAV-939         249
Momelotinib     249
Cediranib       248
Obatoclax       195
Luminespib      194
Alvespimycin     97
Name: count, dtype: int64

In [17]:
adata[adata.obs.split == 'test'].obs.condition.value_counts()

condition
control         664
ENMD-2076       280
MK-0752         202
RG108           196
Ramelteon       195
               ... 
Flavopiridol     68
Luminespib       68
Patupilone       65
Epothilone       56
YM155            40
Name: count, Length: 188, dtype: int64

Also a split which sees all data:

In [19]:
adata.obs['split_all'] = 'train'
test_idx = sc.pp.subsample(adata, .10, copy=True).obs.index
adata.obs['split_all'].loc[test_idx] = 'test'

In [20]:
adata.obs['ct_dose'] = adata.obs.cell_type.astype('str') + '_' + adata.obs.dose_val.astype('str')

Round robin splits: dose and cell line combinations will be held out in turn.

In [21]:
i = 0
split_dict = {}

In [22]:
# single ct holdout
for ct in adata.obs.cell_type.unique():
    for dose in adata.obs.dose_val.unique():
        i += 1
        split_name = f'split{i}'
        split_dict[split_name] = f'{ct}_{dose}'
        
        adata.obs[split_name] = 'train'
        adata.obs[split_name][adata.obs.ct_dose == f'{ct}_{dose}'] = 'ood'
        
        test_idx = sc.pp.subsample(adata[adata.obs[split_name] != 'ood'], .16, copy=True).obs.index
        adata.obs[split_name].loc[test_idx] = 'test'
        
        display(adata.obs[split_name].value_counts())

split1
train    229595
test      43732
ood       17561
Name: count, dtype: int64

split2
train    228515
test      43526
ood       18847
Name: count, dtype: int64

split3
train    229387
test      43692
ood       17809
Name: count, dtype: int64

split4
train    231132
test      44025
ood       15731
Name: count, dtype: int64

split5
train    242945
test      46275
ood        1668
Name: count, dtype: int64

...

split11
train    229287
test      43673
ood       17928
Name: count, dtype: int64

split12
train    227678
test      43367
ood       19843
Name: count, dtype: int64

split13
train    228686
test      43559
ood       18643
Name: count, dtype: int64

split14
train    231557
test      44105
ood       15226
Name: count, dtype: int64

split15
train    242928
test      46272
ood        1688
Name: count, dtype: int64

In [23]:
# double ct holdout
for cts in [('A549', 'MCF7'), ('A549', 'K562'), ('MCF7', 'K562')]:
    for dose in adata.obs.dose_val.unique():
        i += 1
        split_name = f'split{i}'
        split_dict[split_name] = f'{cts[0]}+{cts[1]}_{dose}'
        
        adata.obs[split_name] = 'train'
        adata.obs[split_name][adata.obs.ct_dose == f'{cts[0]}_{dose}'] = 'ood'
        adata.obs[split_name][adata.obs.ct_dose == f'{cts[1]}_{dose}'] = 'ood'
        
        test_idx = sc.pp.subsample(adata[adata.obs[split_name] != 'ood'], .16, copy=True).obs.index
        adata.obs[split_name].loc[test_idx] = 'test'
        
        display(adata.obs[split_name].value_counts())

split16
train    199842
ood       52982
test      38064
Name: count, dtype: int64

split17
train    196536
ood       56917
test      37435
Name: count, dtype: int64

split18
train    198053
ood       55111
test      37724
Name: count, dtype: int64

split19
train    204216
ood       47774
test      38898
Name: count, dtype: int64

split20
train    240335
test      45777
ood        4776
Name: count, dtype: int64

...

split26
train    199533
ood       53349
test      38006
Name: count, dtype: int64

split27
train    195699
ood       57913
test      37276
Name: count, dtype: int64

split28
train    197353
ood       55945
test      37590
Name: count, dtype: int64

split29
train    204640
ood       47269
test      38979
Name: count, dtype: int64

split30
train    240318
test      45774
ood        4796
Name: count, dtype: int64

In [21]:
# triple ct holdout
for dose in adata.obs.dose_val.unique():
    i += 1
    split_name = f'split{i}'

    split_dict[split_name] = f'all_{dose}'
    adata.obs[split_name] = 'train'
    adata.obs[split_name][adata.obs.dose_val == dose] = 'ood'

    test_idx = sc.pp.subsample(adata[adata.obs[split_name] != 'ood'], .16, copy=True).obs.index
    adata.obs[split_name].loc[test_idx] = 'test'

    display(adata.obs[split_name].value_counts())

<ipython-input-21-58d929ed9378>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs[split_name][adata.obs.dose_val == dose] = 'ood'
/home/icb/carlo.dedonno/anaconda3/envs/cpi2/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/home/icb/carlo.dedonno/anaconda3/envs/cpi2/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


train    184782
ood       70910
test      35196
Name: split25, dtype: int64

<ipython-input-21-58d929ed9378>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs[split_name][adata.obs.dose_val == dose] = 'ood'
/home/icb/carlo.dedonno/anaconda3/envs/cpi2/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/home/icb/carlo.dedonno/anaconda3/envs/cpi2/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


train    179868
ood       76760
test      34260
Name: split26, dtype: int64

<ipython-input-21-58d929ed9378>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs[split_name][adata.obs.dose_val == dose] = 'ood'
/home/icb/carlo.dedonno/anaconda3/envs/cpi2/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/home/icb/carlo.dedonno/anaconda3/envs/cpi2/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


train    182393
ood       73754
test      34741
Name: split27, dtype: int64

<ipython-input-21-58d929ed9378>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs[split_name][adata.obs.dose_val == dose] = 'ood'
/home/icb/carlo.dedonno/anaconda3/envs/cpi2/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/home/icb/carlo.dedonno/anaconda3/envs/cpi2/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


train    185997
ood       69464
test      35427
Name: split28, dtype: int64

In [22]:
adata.uns['splits'] = split_dict

In [24]:
sc.write('./datasets/sciplex3_new.h5ad', adata)